First, Select GPU in the runtime session.

Change environment:

In [ ]:
!sed -i 's/np.complex/np.complex128/g' /usr/local/lib/python3.10/dist-packages/librosa/core/constantq.py

import necessary requirements:

In [ ]:
%pip install --upgrade pip
%pip uninstall -y tensorflow numba
%pip install matplotlib\
numpy==1.19.3 \
inflect==0.2.5 \
librosa==0.6.0 \
Unidecode==1.0.22 \
pillow
%pip install janome==0.4.2
%pip install denoiser==0.1.0
%pip install librosa==0.8.0
%pip install numba
%pip install pysoundfile==0.9.0.post1
%pip install unidecode==1.3.4
%pip install pyopenjtalk==0.2.0
%pip install inflect==5.6.2
%pip install janome==0.4.2

Clone repo from github:

In [ ]:
!git clone https://github.com/XeuphoriaXL/CSCI596-Final-Project.git
!cp -r /content/CSCI596-Final-Project/* /content

Download pre-trained model:

In [ ]:
!gdown --folder "https://drive.google.com/drive/folders/1ql2jv4JTL7klWXQTMSnJmrMNMc1aRU5Q?usp=drive_link" -O /content
#!cp -r /content/tacotron2-japanese/* /content

In [ ]:
!pip install pyopenjtalk
import matplotlib
%matplotlib inline
import matplotlib.pylab as plt
import IPython.display as ipd
import sys
sys.path.append('waveglow/')
import numpy as np
import torch
import time
import tracemalloc
from torch.profiler import profile, record_function, ProfilerActivity
import librosa             # MODIFICATION
import librosa.display
import matplotlib.pyplot as plt

After the error, run the below cell and then run the upper cell again.

In [ ]:
!sed -i 's/np.complex/np.complex128/g' /usr/local/lib/python3.10/dist-packages/librosa/core/constantq.py

In [ ]:
from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT, STFT
from audio_processing import griffin_lim
from train import load_model
from text import text_to_sequence
from waveglow.denoiser import Denoiser

In [ ]:
def plot_data(data, figsize=(16, 4)):
    """
    plot function to measure vectorized data distribution
    """
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='lower',
                       interpolation='none')

In [ ]:
def measure_time_and_memory(func, *args, **kwargs):
    """
    Utility function to measure time and memory usage for a function.
    """
    tracemalloc.start()
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    print(f"Execution time: {end_time - start_time:.2f}s")
    print(f"Current memory usage: {current / 10 ** 6:.2f}MB; Peak: {peak / 10 ** 6:.2f}MB")
    return result

In [ ]:
hparams = create_hparams()
hparams.sampling_rate = 22050

In [ ]:
#ayachi_nene_1, ayachi_nene_3, and other modules under content also available
checkpoint_path = "ayachi_nene_2"
model = load_model(hparams)
#load voice model into Cuda memory
model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
_ = model.cuda().eval()#.half()

In [ ]:
waveglow_path = 'waveglow_256channels_universal_v5.pt'
waveglow = torch.load(waveglow_path)['model']
#load waveglow model into Cuda memory
waveglow.cuda().eval()#.half()
for k in waveglow.convinv:
    k.float()
denoiser = Denoiser(waveglow)

In [ ]:
#replace text with anything you want in hiragana or katakana; examples in readme
text = "こんにちは。あやちねねです。どうぞよろしくお願いいたします。"
sequence = np.array(text_to_sequence(text, ['japanese_cleaners']))[None, :]
#load sequence in Cuda
sequence = torch.autograd.Variable(
    torch.from_numpy(sequence)).cuda().long()

In [ ]:
# Measure performance of model inference
mel_outputs, mel_outputs_postnet, _, alignments = measure_time_and_memory(
    model.inference, sequence
)
plot_data((mel_outputs.float().data.cpu().numpy()[0],
           mel_outputs_postnet.float().data.cpu().numpy()[0],
           alignments.float().data.cpu().numpy()[0].T))

In [ ]:
# Measure performance of WaveGlow audio generation
with torch.no_grad():
    audio = measure_time_and_memory(waveglow.infer, mel_outputs_postnet, sigma=0.666)
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

In [ ]:
#plot waveglow 
%matplotlib inline
import matplotlib.pyplot as plt

audio_data = audio[0].data.cpu().numpy()
plt.figure(figsize=(14,4))
plt.plot(audio_data)
plt.title("Generated Audio Waveform")
plt.xlabel("Sample Index")
plt.ylabel("Amplitude")
plt.show()  # Make sure this is included

Run this cell if you wish to download the audio

In [ ]:
import soundfile as sf
from google.colab import files

# Extract the audio data as a NumPy array
audio_np = audio[0].data.cpu().numpy()

# Write it to a WAV file (16-bit PCM)
sf.write('generated_audio.wav', audio_np, hparams.sampling_rate, subtype='PCM_16')

# Download the file
files.download('generated_audio.wav')